# Graph Convolutional Network

This notebook demonstrates the training of [Graph Convolutional Networks (GCN)](https://arxiv.org/pdf/1609.02907.pdf) with TigerGraph. We implement a GCN using the [Spektral](https://graphneural.network/) GNN framework. We train the model on the Cora dataset with TigerGraph as the data store. The dataset contains 2708 machine learning papers and 10556 citation links between the papers. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from a dictionary. The dictionary consists of 1433 unique words. Each paper is classified into one of seven classes based on the topic. The goal is to predict the class of each vertex in the graph.

The following libraries are required to run this notebook. Uncomment to install them if necessary. You might need to restart the kernel after installing.


In [ ]:
#!pip install tensorflow
#!pip install spektral==1.2.0
#!pip install pyTigerGraph[gds]

## Table of Contents
* [Data Processing](#data_processing)  
* [Train on whole graph](#train_whole)  
* [Train on neighborhood subgraphs](#train_subgraph) 
* [Inference](#inference) 

## Data Processing <a name="data_processing"></a>

### Connect to TigerGraph

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details. 

To connect your database, modify the `config.json` file accompanying this notebook. Set the value of `getToken` based on whether token auth is enabled for your database. Token auth is always enabled for tgcloud databases. 

In [ ]:
from pyTigerGraph import TigerGraphConnection
import json

# Read in DB configs
with open('../../config.json', "r") as config_file:
    config = json.load(config_file)
    
conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"]
)

### Ingest Data

In [ ]:
from pyTigerGraph.datasets import Datasets

dataset = Datasets("Cora")

In [ ]:
conn.ingestDataset(dataset, getToken=config["getToken"])

### Visualize Schema

In [ ]:
from pyTigerGraph.visualization import drawSchema

drawSchema(conn.getSchema(force=True))

### Basic Statistics

In [ ]:
conn.getVertexCount('*')

In [ ]:
conn.getEdgeCount()

### Train/validation/test split

In [ ]:
# The code in this cell is commented out because there is no need to split the vertices into 
# training/validation/test sets, as the split is already done in the original dataset. 
# See notebook 1_data_processing for examples on the split function.

#split = conn.gds.vertexSplitter(train_mask=0.8, val_mask=0.1, test_mask=0.1)
#split.run()

In [ ]:
print(
    "Number of vertices in training set:",
    conn.getVertexCount("Paper", where="train_mask!=0"),
)
print(
    "Number of vertices in validation set:",
    conn.getVertexCount("Paper", where="val_mask!=0"),
)
print(
    "Number of vertices in test set:", 
    conn.getVertexCount("Paper", where="test_mask!=0"),
)

## Train on whole graph <a name="train_whole"></a>
We first train the model on the whole graph. This will **NOT** work when the graph is large. See the section of training on subgraphs for real use. However, we still include this example for illustration purpose. Hyperparameters for the model and training environment are defined below.

In [ ]:
# Hyperparameters
hp = {"hidden_dim": 64, 
      "num_layers": 2, 
      "dropout": 0.6,
      "lr": 0.001, 
      "l2_penalty": 5e-4}

### Construct graph loader

The `GraphLoader` can get the whole graph from database all at once (`num_batches=1`). See the tutorial on dataloaders for details.

In [ ]:
graph_loader = conn.gds.graphLoader(
    v_in_feats=["x"],
    v_out_labels=["y"],
    v_extra_feats=["train_mask", "val_mask", "test_mask"],
    num_batches=1,
    output_format="spektral",
    shuffle=False
)

In [ ]:
# Get the whole graph from the loader in PyG format
data = graph_loader.data

data

In [ ]:
x, adj, y, mask_tr, mask_va, mask_te = data.x, data.A, data.y, data.train_mask, data.val_mask, data.test_mask

### Construct model and optimizer

We build a GCN model with 2 convolutional layers, and use the Adam optimizer with a learning rate of 0.01.

In [ ]:
import numpy as np

from spektral.layers import GCNConv
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

from spektral.layers import GraphSageConv

In [ ]:
device = tf.device("GPU" if tf.config.list_physical_devices('GPU') else "CPU")

# Model definition
x_in = Input(shape=(data.n_node_features,))
a_in = Input(shape=(None,), sparse=True)
x_1 = GCNConv(hp["hidden_dim"], activation="relu")([x_in, a_in])
x_1 = Dropout(hp["dropout"])(x_1)
x_2 = GCNConv(7, activation="softmax")([x_1, a_in])

# Build model
model = Model(inputs=[x_in, a_in], outputs=x_2)
model.summary()

In [ ]:
optimizer = Adam(learning_rate=hp["lr"])
loss_fn = CategoricalCrossentropy()

one_hot_y = to_categorical(y, num_classes=7)
tf_a = tf.SparseTensor(#converts the scipy sparse matrix to a tensorflow sparse matrix
    indices=np.array([adj.row, adj.col]).T,
    values=adj.data,
    dense_shape=adj.shape)

### Train the model

In [ ]:
from datetime import datetime

In [ ]:
# Training step
@tf.function
def train():
    with tf.GradientTape() as tape:
        predictions = model([x, tf_a], training=True)
        loss = loss_fn(tf.boolean_mask(one_hot_y, mask_tr), tf.boolean_mask(predictions, mask_tr))
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
@tf.function
def evaluate():
    predictions = model([x, tf_a], training=False)
    losses = []
    accuracies = []
    for mask in [mask_tr, mask_va, mask_te]:
        loss = loss_fn(tf.boolean_mask(one_hot_y, mask), tf.boolean_mask(predictions, mask))
        loss += sum(model.losses)
        losses.append(loss)
        acc = tf.reduce_mean(categorical_accuracy(tf.boolean_mask(one_hot_y, mask), tf.boolean_mask(predictions, mask)))
        accuracies.append(acc)
    return losses, accuracies

In [ ]:
epochs = 20
for epoch in range(1, epochs + 1):
    train()
    l, a = evaluate()
    print(
        "Epoch {:.0f}:\n"
        "Training Loss: {:.4f}, Training Accuracy: {:.4f}, "
        "Validation Loss: {:.4f}, Validation Accuracy: {:.4f}".format(epoch, l[0], a[0], l[1], a[1])
    )

### Test the model

In [ ]:
l, a = evaluate()

print("Testing Loss: {:.4f}, Testing Accuracy: {:.4f}".format(l[2], a[2]))

## Train on Neighborhood Subgraphs <a name="train_subgraph"></a>
Alternatively, we train the model on the neighborhood subgraphs. Each subgraph contains the 2 hop neighborhood of certain seed vertices. This method  will allow us to train the model on graphs that are way larger than the CORA dataset because we don't load the whole graph into memory all at once. 

We will use the same parameters as before, but we will use the NeighborLoader to load subgraphs. Once we finish iterating over all the subgraphs generated by the loader, it is guaranteed to cover all vertices in the graph (except for those filtered by a user provided mask). 

In [ ]:
# Hyperparameters
hp = {"batch_size": 64, 
      "num_neighbors": 10, 
      "num_hops": 2, 
      "hidden_dim": 64,
      "num_layers": 2, 
      "dropout": 0.6, 
      "lr": 0.01, 
      "l2_penalty": 5e-4}

### Construct neighborhood subgraph loader

Here we construct 3 subgraph loaders. The `train_loader` only uses vertices in the training set as seeds, the `valid_loader` only uses vertices in the validation set, and the `test_loader` only uses vertices in the test set.

In [ ]:
train_loader = conn.gds.neighborLoader(
    v_in_feats=["x"],
    v_out_labels=["y"],
    v_extra_feats=["train_mask","val_mask","test_mask"],
    output_format="spektral",
    batch_size=hp["batch_size"],
    num_neighbors=hp["num_neighbors"],
    num_hops=hp["num_hops"],
    shuffle=True,
    filter_by="train_mask",
)

In [ ]:
valid_loader = conn.gds.neighborLoader(
    v_in_feats=["x"],
    v_out_labels=["y"],
    v_extra_feats=["train_mask","val_mask","test_mask"],
    output_format="spektral",
    batch_size=hp["batch_size"],
    num_neighbors=hp["num_neighbors"],
    num_hops=hp["num_hops"],
    shuffle=False,
    filter_by="val_mask",
)

### Construct model and optimizer

We build a graphSAGE model with 2 convolutional layers, and use the Adam optimizer with a learning rate of 0.01.

In [ ]:
model = Model(inputs=[x_in, a_in], outputs=x_2)
model.summary()

In [ ]:
optimizer = Adam(learning_rate=hp["lr"])
loss_fn = CategoricalCrossentropy()

### Train the model

In [ ]:
from datetime import datetime

In [ ]:
def preprocess_batch(graph):
    x, adj, y, mask_tr, mask_va, mask_te = graph.x, graph.A, graph.y, graph.train_mask, graph.val_mask, graph.test_mask
    one_hot_y = to_categorical(y, num_classes=7)
    tf_a = tf.SparseTensor(#converts the scipy sparse matrix to a tensorflow sparse matrix
        indices=np.array([adj.row, adj.col]).T,
        values=adj.data,
        dense_shape=adj.shape)
    return x, tf_a, one_hot_y

val_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_loss_metric = tf.keras.metrics.CategoricalCrossentropy()

In [ ]:
@tf.function
def train(x, tf_a, one_hot_y, mask):
    with tf.GradientTape() as tape:
        predictions = model([x, tf_a], training=True)
        loss = loss_fn(tf.boolean_mask(one_hot_y, mask), tf.boolean_mask(predictions, mask))
        acc = tf.reduce_mean(categorical_accuracy(tf.boolean_mask(one_hot_y, mask), tf.boolean_mask(predictions, mask)))
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, acc

In [ ]:
@tf.function
def test_step(x, tf_a, one_hot_y, mask, metrics=[]):
    val_logits = model([x, tf_a], training=False)
    for metric in metrics:
      metric.update_state(tf.boolean_mask(one_hot_y, mask), tf.boolean_mask(val_logits, mask))

In [ ]:
for epoch in range(10):
    for bid, batch in enumerate(train_loader):
        batchsize = batch.n_nodes
        x, tf_a, one_hot_y = preprocess_batch(batch)
        loss, acc = train(x, tf_a, one_hot_y, batch.train_mask)
        print("Epoch {}, Train Batch {}, Loss {:.4f}, Accuracy {:.4f}".format(epoch, bid, loss, acc))
    for batch in valid_loader:
        x, tf_a, one_hot_y = preprocess_batch(batch)
        test_step(x, tf_a, one_hot_y, batch.val_mask, metrics = [val_acc_metric, val_loss_metric])
    val_acc = val_acc_metric.result()
    val_loss = val_loss_metric.result()
    val_acc_metric.reset_states()
    val_loss_metric.reset_states()
    print("Epoch {}, Valid Loss {:.4f}, Valid Accuracy {:.4f}".format(epoch, val_loss, val_acc))

### Test the model

In [ ]:
test_loader = conn.gds.neighborLoader(
    v_in_feats=["x"],
    v_out_labels=["y"],
    v_extra_feats=["train_mask","val_mask","test_mask"],
    output_format="spektral",
    batch_size=hp["batch_size"],
    num_neighbors=hp["num_neighbors"],
    num_hops=hp["num_hops"],
    shuffle=False,
    filter_by="test_mask",
)

In [ ]:
acc = tf.keras.metrics.CategoricalAccuracy()
for batch in test_loader:
    x, tf_a, one_hot_y = preprocess_batch(batch)
    test_step(x, tf_a, one_hot_y, batch.val_mask, metrics = [acc])
print("Accuracy: {:.4f}".format(acc.result()))
acc.reset_states()

## Inference <a name="inference"></a>

Finally, we use the trained model for node classification. At this stage, we typically do inference/prediction for specific nodes instead of random batches, so we will create a new data loader.

In [ ]:
infer_loader = conn.gds.neighborLoader(
    v_in_feats=["x"],
    v_out_labels=["y"],
    v_extra_feats=["train_mask","val_mask","test_mask"],
    output_format="spektral",
    num_neighbors=hp["num_neighbors"],
    num_hops=hp["num_hops"],
    shuffle=False,
)

In [ ]:
# Fetch specific nodes by their IDs and do prediction. 
# Each node is represented by a dict with two mandatory keys: primary_id and type.
input_nodes = [{"primary_id": 7, "type": "Paper"}, 
               {"primary_id": 999, "type": "Paper"}]
data = infer_loader.fetch(input_nodes)

In [ ]:
# The returned data are the neighborhood subgraphs of the input nodes.
# The original IDs of the nodes in the subgraphs are stored in the 
# `primary_id` attribute.
data

In [ ]:
# Predict. Predictions for both the input nodes and others in their 
# neighborhoods are generated.
x, tf_a, one_hot_y = preprocess_batch(batch)
pred = model([x, tf_a], training=False)
print("ID: Label")
for i,j in zip(data.primary_id, pred):
    print("{}:{}".format(i, tf.math.argmax(j)))